In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
test["date_block_num"] = 34

train_gp = train.groupby(["date_block_num","shop_id","item_id"]).agg({"item_cnt_day":["sum"]})
train_gp.columns = ["item_cnt_month"]
train_gp.reset_index(inplace=True)

# train = pd.merge(train,train_gp,on=["date_block_num","shop_id","item_id"],how="left")
# train = train.drop(["date","item_price","item_cnt_day"],axis=1)

In [ ]:
last_month = train_gp.copy()
last_month["date_block_num"] = last_month["date_block_num"] + 1
last_month = last_month.rename(columns={'item_cnt_month':'last_month_cnt'})
train_gp = pd.merge(train_gp,last_month,on=["date_block_num","shop_id","item_id"],how="left")
train_gp = train_gp[train_gp.date_block_num!=0].fillna(0).reset_index(drop=True)

In [ ]:
test = pd.merge(test,last_month,on=["date_block_num","shop_id","item_id"],how="left")
test = test.fillna(0).reset_index(drop=True)

In [ ]:
y_true = np.where(train_gp.query("date_block_num == 33")['item_cnt_month']>20,20,train_gp.query("date_block_num == 33")['item_cnt_month'])
y_pred = np.where(train_gp.query("date_block_num == 33")['last_month_cnt']>20,20,train_gp.query("date_block_num == 33")['last_month_cnt'])

In [ ]:
mean_squared_error(y_true,y_pred,squared=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
fea = ['last_month_cnt','shop_id','item_id']
train_gp.item_cnt_month = np.where(train_gp.item_cnt_month>20,20,train_gp.item_cnt_month)
X_train = train_gp.query("date_block_num < 33")[fea]
X_test = train_gp.query("date_block_num == 33")[fea]
y_train = train_gp.query("date_block_num < 33")['item_cnt_month']
y_test = train_gp.query("date_block_num == 33")['item_cnt_month']

In [ ]:
rfr = RandomForestRegressor(max_depth=10,n_jobs=-1)
rfr.fit(X_train,y_train)

In [ ]:
y_pred = rfr.predict(X_test)

In [ ]:
y_true = np.where(train_gp.query("date_block_num == 33")['item_cnt_month']>20,20,train_gp.query("date_block_num == 33")['item_cnt_month'])
y_pred = np.where(y_pred>20,20,y_pred)

In [ ]:
mean_squared_error(y_true,y_pred,squared=False)

In [ ]:
test['pred'] = rfr.predict(test[fea])
test['pred'] = np.where(test['pred']>20,20,test['pred'])


In [ ]:
test

In [ ]:
test.loc[test['last_month_cnt']==0,'pred']= 0 

In [ ]:
output = pd.DataFrame({"ID":test["ID"],"item_cnt_month":test['pred']})
print(output.head())
output.to_csv("output.csv",index=False)